In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv


In [55]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset,DatasetDict,Features,Value,ClassLabel
# Load the dataset
df = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')

# Split the data into features (X) and target (y)
X = df.drop('sentiment', axis=1)
y = df['sentiment']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the validation set into validation and test sets
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42)

# Combine the features and target back into DataFrames
train_df = pd.concat([X_train, y_train], axis=1)
val_df = pd.concat([X_val, y_val], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

# Convert the DataFrames to Dataset instances
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Display the first few rows of the training dataset
# print(train_dataset[:5])

In [56]:
dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment', '__index_level_0__'],
        num_rows: 40000
    })
    validation: Dataset({
        features: ['review', 'sentiment', '__index_level_0__'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['review', 'sentiment', '__index_level_0__'],
        num_rows: 5000
    })
})

In [50]:
dataset_dict.set_format(type="pandas")
df = dataset_dict["train"][:]
df.head()

,review,sentiment,__index_level_0__
0,That's what I kept asking myself during the ma...,negative,39087
1,I did not watch the entire movie. I could not ...,negative,30893
2,A touching love story reminiscent of In the M...,positive,45278
3,This latter-day Fulci schlocker is a totally a...,negative,16398
4,"First of all, I firmly believe that Norwegian ...",negative,13653


In [85]:
import torch
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning:

`clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884



In [86]:
def tokenize_and_embed(batch):
    # Tokenize the text
    inputs = tokenizer(batch['review'], padding=True, truncation=True, max_length=512, return_tensors='pt')
    
    # Move the input tensors to the GPU
    inputs = {key: val.to(device) for key, val in inputs.items()}
    
    # Get the embeddings
    with torch.no_grad():
        model_output = model(**inputs)
    embeddings = model_output.pooler_output  # Use the [CLS] token as the embedding
    
    # Move the embeddings back to the CPU and convert to a NumPy array
    embeddings = embeddings.cpu().numpy()
    
    # Ensure the embeddings are returned as a list of arrays
    return {'embedding': embeddings.tolist()}

def binary_encode_sentiment(batch):
    # Convert sentiment to binary encoding (1 for positive, 0 for negative)
    sentiment_map = {'positive': 1, 'negative': 0}
    batch['sentiment'] = [sentiment_map[sent] for sent in batch['sentiment']]
    return {'sentiment': batch['sentiment']}

In [89]:
dataset_dict["train"] = dataset_dict["train"].map(tokenize_and_embed, batched=True, batch_size=128)
dataset_dict["train"] = dataset_dict["train"].map(binary_encode_sentiment, batched=True, batch_size=128)
dataset_dict["train"] = dataset_dict["train"].remove_columns(['review', '__index_level_0__'])
print(dataset_dict["train"][0])
dataset_dict["test"] = dataset_dict["test"].map(tokenize_and_embed, batched=True, batch_size=128)
dataset_dict["test"] = dataset_dict["test"].map(binary_encode_sentiment, batched=True, batch_size=128)
dataset_dict["test"] = dataset_dict["test"].remove_columns(['review', '__index_level_0__'])
print(dataset_dict["test"][0])
dataset_dict["validation"] = dataset_dict["validation"].map(tokenize_and_embed, batched=True, batch_size=128)
dataset_dict["validation"] = dataset_dict["validation"].map(binary_encode_sentiment, batched=True, batch_size=128)
dataset_dict["validation"] = dataset_dict["validation"].remove_columns(['review', '__index_level_0__'])
print(dataset_dict["validation"][0])

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'sentiment': 0, 'embedding': [-0.6027631759643555, 0.016370631754398346, -0.3594742715358734, -0.42547786235809326, 0.5641049742698669, -0.5599179267883301, -0.09022077172994614, -0.4565143585205078, -0.19502544403076172, -0.713045597076416, -0.20133620500564575, -0.5266204476356506, 0.46570277214050293, -0.4904267489910126, 0.5933690667152405, 0.4105309844017029, 0.771356463432312, 0.30532947182655334, -0.1621474176645279, 0.7986869215965271, -0.25509411096572876, -0.7410961389541626, -0.2667204439640045, -0.1377800852060318, -0.7987908124923706, 0.005792377516627312, -0.4875742793083191, 0.22282402217388153, -0.7780532836914062, 0.09671440720558167, 0.16834355890750885, -0.3213283121585846, -0.0853966772556305, -0.8645196557044983, -0.19212739169597626, -0.018226031213998795, 0.10945405811071396, 0.0015858201077207923, 0.477262407541275, -0.6413108110427856, 0.6022481918334961, -0.9431482553482056, -0.5729193687438965, 0.472118616104126, -0.6088852882385254, -0.017115041613578796, -

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

{'sentiment': 0, 'embedding': [-0.47350484132766724, -0.1000252515077591, 0.1837751269340515, -0.4410695731639862, -0.09055770188570023, -0.5882434844970703, -0.5168480277061462, -0.3233974575996399, -0.05567567050457001, -0.3852311074733734, -0.15609660744667053, 0.17608900368213654, -0.2636236250400543, -0.028037989512085915, 0.2868318557739258, 0.042375460267066956, 0.7189849615097046, 0.09365946799516678, -0.3727757930755615, 0.5142138600349426, -0.15649016201496124, -0.13132353127002716, -0.22718502581119537, -0.08395645767450333, -0.23081031441688538, -0.32022854685783386, 0.03257125988602638, 0.05419398471713066, -0.27151575684547424, -0.17791201174259186, 0.7555751800537109, 0.297224760055542, -0.20007649064064026, -0.6727972030639648, 0.27439823746681213, 0.4832115173339844, -0.1820460706949234, -0.24254541099071503, -0.01766449399292469, -0.650313675403595, -0.3636385500431061, -0.49168166518211365, -0.38231855630874634, -0.5550156235694885, -0.4661543071269989, -0.2044426500

In [144]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Convert the dataset to PyTorch tensors
train_tensors = TensorDataset(
    torch.tensor(dataset_dict["train"]['embedding'], dtype=torch.float32),
    torch.tensor(dataset_dict["train"]['sentiment'], dtype=torch.long)
)

test_tensors = TensorDataset(
    torch.tensor(dataset_dict["test"]['embedding'], dtype=torch.float32),
    torch.tensor(dataset_dict["test"]['sentiment'], dtype=torch.long)
)

validation_tensors = TensorDataset(
    torch.tensor(dataset_dict["validation"]['embedding'], dtype=torch.float32),
    torch.tensor(dataset_dict["validation"]['sentiment'], dtype=torch.long)
)

# Create the dataloader
batch_size = 16
train_dataloader = DataLoader(train_tensors, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_tensors, batch_size=batch_size, shuffle=True)
validation_dataloader = DataLoader(validation_tensors, batch_size=batch_size, shuffle=True)



In [145]:
class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.sequential = nn.Sequential(
            nn.Linear(768, 256),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.Dropout(0.1),
            nn.ReLU(),
            nn.Linear(32, 1),
        )

    def forward(self, x):
        x = self.sequential(x)
        return x

In [146]:
from torch.optim import Adam
# from torch.optim import lr_scheduler

epochs = 10
model = BinaryClassifier().to(device)
optimizer = Adam(model.parameters(), lr=0.0003)
loss_fn = nn.BCEWithLogitsLoss()

In [147]:
from tqdm.autonotebook import tqdm
import torch.nn.functional as F

def train_one_epoch(
    model,
    loader,
    optimizer,
    loss_fn,
    epoch_num=-1
):
    loop = tqdm(
        enumerate(loader, 1),
        total=len(loader),
        desc=f"Epoch {epoch_num}: train",
        leave=True,
    )
    model.train()
    train_loss = 0.0
    for i, batch in loop:
        embeddings, labels = batch
        embeddings = embeddings.to(device)
        labels = labels.view(-1, 1).to(torch.float).to(device)
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward pass
        outputs = model(embeddings)
        
        # loss calculation
        loss = loss_fn(outputs, labels)
        
        # backward pass
        loss.backward()

        # optimizer run
        optimizer.step()
        

        train_loss += loss.item()
        loop.set_postfix({"loss": train_loss/(i * len(labels))})

def val_one_epoch(
    model,
    loader,
    loss_fn,
    epoch_num=-1,
    best_so_far=0.0,
    ckpt_path='best.pt'
):
    
    loop = tqdm(
        enumerate(loader, 1),
        total=len(loader),
        desc=f"Epoch {epoch_num}: val",
        leave=True,
    )
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        model.eval()  # evaluation mode
        for i, batch in loop:
            embeddings, labels = batch
            embeddings = embeddings.to(device)
            labels = labels.view(-1, 1).to(torch.float).to(device)

            # forward pass
            outputs = model(embeddings)
            
            # loss calculation
            loss = loss_fn(outputs, labels)
            
            outputs = torch.round(F.sigmoid(outputs))
            
            total += labels.size(0)
            
            correct += (outputs == labels).sum().item()

            val_loss += loss.item()
            loop.set_postfix({"loss": val_loss/total, "acc": correct / total})

        if correct / total > best_so_far:
            torch.save(model.state_dict(), ckpt_path)
            return correct / total

    return best_so_far

In [148]:
best = -float('inf')
for epoch in range(epochs):
    train_one_epoch(model, train_dataloader, optimizer, loss_fn, epoch_num=epoch)
    best = val_one_epoch(model, validation_dataloader, loss_fn, epoch, best_so_far=best)

Epoch 0: train:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 0: val:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch 1: train:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 1: val:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch 2: train:   0%|          | 0/2500 [00:00<?, ?it/s]

...

Epoch 7: val:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch 8: train:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 8: val:   0%|          | 0/313 [00:00<?, ?it/s]

Epoch 9: train:   0%|          | 0/2500 [00:00<?, ?it/s]

Epoch 9: val:   0%|          | 0/313 [00:00<?, ?it/s]

In [153]:
def predict(
    model,
    loader,
):
    loop = tqdm(
        enumerate(loader, 1),
        total=len(loader),
        desc="Predictions:",
        leave=True,
    )
    correct = 0
    total = 0
    predictions = []
    with torch.no_grad():
        model.eval()  # evaluation mode
        for i, batch in loop:
            embeddings, labels = batch
            embeddings = embeddings.to(device)
            labels = labels.view(-1, 1).to(torch.float).to(device)

            # forward pass
            outputs = model(embeddings)
            
            predictions = torch.round(F.sigmoid(outputs))
            
            total += labels.size(0)
            
            correct += (outputs == labels).sum().item()

            loop.set_postfix({"acc": correct / total})

    return predictions

In [154]:
ckpt = torch.load("best.pt")
model.load_state_dict(ckpt)

predictions = predict(model, test_dataloader)
predictions[:10]

/tmp/ipykernel_30/305245208.py:1: FutureWarning:

You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.



Predictions::   0%|          | 0/313 [00:00<?, ?it/s]

[]